In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset
from torchvision import datasets, models
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import pandas as pd

from torchvision import transforms, utils
from torch.utils.data import DataLoader
from skimage import io, transform

import glob
import re

import cv2 as cv


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")

In [ ]:
train

In [ ]:
y=train['Pawpularity']
y

In [ ]:
class Pet_Data(Dataset):

    def __init__(self, path=None, transform=None):
        self.transform = transform
        self.path = path
        self.img_path = [] 
        for dirname, _, filenames in os.walk(self.path):
            for filename in filenames:
                self.img_path.append(os.path.join(dirname, filename))

        self.transform = transform

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        img_name = os.path.splitext(os.path.relpath(self.img_path[idx],self.path))[0]
#         print(img_name)
        i = np.where((train['Id'] == img_name))
#         print(f" i:{i[0][0]}")
        i = i[0][0]
        pwp = train['Pawpularity'][i] 
#         pwp = pwp/100
        image = io.imread(self.img_path[idx])

        if self.transform:
            trns_image = self.transform(image)
        
        sample = {'image': trns_image, 'name': img_name, 'pwp':pwp}
        return sample

In [ ]:
# class Rescale_Img(object):
#     def __init__(self, output_size):
#         self.output_size = int(output_size)
#         print(self.output_size)

#     def __call__(self, sample):
#         image,name, pwp =sample['image'], sample['name'], sample['pwp']
#         img = transform.resize(image, (self.output_size, self.output_size,3))
#         return {'image': img, 'name': name, 'pwp':pwp}
    


# class Normalize_Img(object):
#     def __call__(self,sample):
#         image,name,pwp= sample['image'],sample['name'],sample['pwp']
# #         mn = np.min(image)
# #         mx = np.max(image)
# #         img = (image - mn) * (1.0 / (mx - mn) )
# #         img = cv.imread(image)
#         norm_img = np.zeros((256,256))
#         final_img = cv.normalize(image,  norm_img, 0, 1, cv.NORM_MINMAX)
# #         print(f"img: {image} \n norm: {final_img}")
#         return {'image': final_img, 'name': name, 'pwp':pwp}


# class ToTensor(object):
#     """Convert ndarrays in sample to Tensors."""

#     def __call__(self, sample):
#         image,name,pwp= sample['image'],sample['name'],sample['pwp']

#         # swap color axis because
#         # numpy image: H x W x C
#         # torch image: C x H x W
#         image = torch.from_numpy(image.transpose((2, 0, 1)))
#         print(f"type: {type(image)}")
#         return {'image': image,
#                 'name': name,
#                 'pwp':pwp}

In [ ]:
# train_data = Pet_Data(path='/kaggle/input/petfinder-pawpularity-score/train/',transform = transforms.Compose([Rescale_Img(256),Normalize_Img(),ToTensor() ]) )
# test_data = Pet_Data(path='/kaggle/input/petfinder-pawpularity-score/test/',transform = transforms.Compose([Rescale_Img(256), Normalize_Img(),ToTensor(), ]) )

MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

train_data = Pet_Data(path='/kaggle/input/petfinder-pawpularity-score/train/', transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((512,512)),
    transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(90),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD),

]) )

test_data = Pet_Data(path='/kaggle/input/petfinder-pawpularity-score/test/', transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((512,512)),
    transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(90),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD),
]) )

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
len(train_data)

In [ ]:
# train_data[2]

In [ ]:
for i in range (10):
    print(f"val : {train_data[i]['image'].shape}  id: {train_data[i]['name']}  pwp: {train_data[i]['pwp']}")

In [ ]:
# class CNN_Net(nn.Module):
    
#     def __init__(self):
#         super(CNN_Net,self).__init__()
#         self.model_layers =nn.Sequential(
            
#             nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, stride=1),
#             nn.ReLU(),

# #             nn.MaxPool2d(kernel_size=2, stride=2),
            
#             nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3),
#             nn.ReLU(),

#             nn.MaxPool2d(kernel_size=2, stride=2),
            
#             nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1),
#             nn.ReLU(),
            
#             nn.MaxPool2d(kernel_size=2, stride=2),
            
#             nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5, stride=2),
#             nn.ReLU(),
#             nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=2),
#             nn.ReLU(),
            
#             nn.MaxPool2d(kernel_size=2, stride=2),
            
#             nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1),
#             nn.ReLU(),
               
#             nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1),
#             nn.ReLU(),
            
#             nn.Flatten(),
            
#             nn.Linear(4608,2024),
#             nn.ReLU(),
#             nn.Linear(2024,100),
#             nn.ReLU(),
#             nn.Linear(100,10),
#             nn.ReLU(),

#             nn.Linear(10,1),

#         )
    
    
#     def forward(self,x):
#         x = self.model_layers(x)
#         return x
    

In [ ]:
# model = CNN_Net().to(device)
# model=model.float()

Manual Resnet Implementation

source to original [github](https://github.com/FrancescoSaverioZuppichini/ResNet/blob/master/ResNet.ipynb)

In [ ]:
import torch
import torch.nn as nn

from functools import partial
from dataclasses import dataclass
from collections import OrderedDict

In [ ]:
class Conv2dAuto(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding =  (self.kernel_size[0] // 2, self.kernel_size[1] // 2) # dynamic add padding based on the kernel_size
        
conv3x3 = partial(Conv2dAuto, kernel_size=3, bias=False) 

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.in_channels, self.out_channels =  in_channels, out_channels
        self.blocks = nn.Identity()
        self.shortcut = nn.Identity()   
    
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut: residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels

In [ ]:
from collections import OrderedDict

class ResNetResidualBlock(ResidualBlock):
    def __init__(self, in_channels, out_channels, expansion=1, downsampling=1, conv=conv3x3, *args, **kwargs):
        super().__init__(in_channels, out_channels)
        self.expansion, self.downsampling, self.conv = expansion, downsampling, conv
        self.shortcut = nn.Sequential(OrderedDict(
        {
            'conv' : nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1,
                      stride=self.downsampling, bias=False),
            'bn' : nn.BatchNorm2d(self.expanded_channels)
            
        })) if self.should_apply_shortcut else None
        
        
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels

In [ ]:
from collections import OrderedDict
def conv_bn(in_channels, out_channels, conv, *args, **kwargs):
    return nn.Sequential(OrderedDict({'conv': conv(in_channels, out_channels, *args, **kwargs), 
                          'bn': nn.BatchNorm2d(out_channels) }))

In [ ]:
class ResNetBasicBlock(ResNetResidualBlock):
    expansion = 1
    def __init__(self, in_channels, out_channels, activation=nn.ReLU, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.blocks = nn.Sequential(
            conv_bn(self.in_channels, self.out_channels, conv=self.conv, bias=False, stride=self.downsampling),
            activation(),
            conv_bn(self.out_channels, self.expanded_channels, conv=self.conv, bias=False),
        )
    

In [ ]:
class ResNetBottleNeckBlock(ResNetResidualBlock):
    expansion = 4
    def __init__(self, in_channels, out_channels, activation=nn.ReLU, *args, **kwargs):
        super().__init__(in_channels, out_channels, expansion=4, *args, **kwargs)
        self.blocks = nn.Sequential(
           conv_bn(self.in_channels, self.out_channels, self.conv, kernel_size=1),
             activation(),
             conv_bn(self.out_channels, self.out_channels, self.conv, kernel_size=3, stride=self.downsampling),
             activation(),
             conv_bn(self.out_channels, self.expanded_channels, self.conv, kernel_size=1),
        )
    

In [ ]:
class ResNetLayer(nn.Module):
    def __init__(self, in_channels, out_channels, block=ResNetBasicBlock, n=1, *args, **kwargs):
        super().__init__()
        # 'We perform downsampling directly by convolutional layers that have a stride of 2.'
        downsampling = 2 if in_channels != out_channels else 1
        
        self.blocks = nn.Sequential(
            block(in_channels , out_channels, *args, **kwargs, downsampling=downsampling),
            *[block(out_channels * block.expansion, 
                    out_channels, downsampling=1, *args, **kwargs) for _ in range(n - 1)]
        )

    def forward(self, x):
        x = self.blocks(x)
        return x

In [ ]:
class ResNetEncoder(nn.Module):
    """
    ResNet encoder composed by increasing different layers with increasing features.
    """
    def __init__(self, in_channels=3, blocks_sizes=[64, 128, 256, 512], deepths=[2,2,2,2], 
                 activation=nn.ReLU, block=ResNetBasicBlock, *args,**kwargs):
        super().__init__()
        
        self.blocks_sizes = blocks_sizes
        
        self.gate = nn.Sequential(
            nn.Conv2d(in_channels, self.blocks_sizes[0], kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(self.blocks_sizes[0]),
            activation(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.in_out_block_sizes = list(zip(blocks_sizes, blocks_sizes[1:]))
        self.blocks = nn.ModuleList([ 
            ResNetLayer(blocks_sizes[0], blocks_sizes[0], n=deepths[0], activation=activation, 
                        block=block,  *args, **kwargs),
            *[ResNetLayer(in_channels * block.expansion, 
                          out_channels, n=n, activation=activation, 
                          block=block, *args, **kwargs) 
              for (in_channels, out_channels), n in zip(self.in_out_block_sizes, deepths[1:])]       
        ])
        
        
    def forward(self, x):
        x = self.gate(x)
        for block in self.blocks:
            x = block(x)
        return x

In [ ]:
class ResnetDecoder(nn.Module):
    """
    This class represents the tail of ResNet. It performs a global pooling and maps the output to the
    correct class by using a fully connected layer.
    """
    def __init__(self, in_features, n_classes):
        super().__init__()
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.decoder = nn.Linear(in_features, n_classes)

    def forward(self, x):
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        x = self.decoder(x)
        return x

In [ ]:
class ResNet(nn.Module):
    
    def __init__(self, in_channels, n_classes, *args, **kwargs):
        super().__init__()
        self.encoder = ResNetEncoder(in_channels, *args, **kwargs)
        self.decoder = ResnetDecoder(self.encoder.blocks[-1].blocks[-1].expanded_channels, n_classes)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
def resnet18(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBasicBlock, deepths=[2, 2, 2, 2])

def resnet34(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBasicBlock, deepths=[3, 4, 6, 3])

def resnet50(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBottleNeckBlock, deepths=[3, 4, 6, 3])

def resnet101(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBottleNeckBlock, deepths=[3, 4, 23, 3])

def resnet152(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBottleNeckBlock, deepths=[3, 8, 36, 3])

Resnet implementaion ends here

In [ ]:
# conv = conv3x3(in_channels=32, out_channels=64)
# print(conv)
# del conv

In [ ]:
class MyModifiedModel(nn.Module):
    def __init__(self, resnet50):
        super(MyModifiedModel,self).__init__()
        self.resnet = resnet50
        self.ln1 = nn.Linear(1000,10)
        self.ln2 = nn.Linear(10,1)

        
    def forward(self, x):
        x = self.resnet(x)
        x = self.ln1(x)
        x = self.ln2(x)

        return x

In [ ]:
resnet50 = resnet18(3, 1000)

for params in resnet50.parameters():
    params.required_grad=False
model = MyModifiedModel(resnet50)
model.to('cuda')

In [ ]:
# !pip install torchsummary

In [ ]:
# from torchsummary import summary
# summary(model,(3,512,512))

In [ ]:
learning_rate = 0.001
batch_size = 16
epochs = 5

In [ ]:
loss_fn = nn.MSELoss()

In [ ]:
# model = model.cuda()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    train_loss =0
    for i,sample in enumerate(dataloader):
        optimizer.zero_grad()
        pred = model(sample['image'].float().cuda())
#         print((sample['image'].float().cuda()))
        true_val = sample['pwp'].float().cuda()
        true_val = true_val.unsqueeze(1)
#         print(f"true Val: {true_val} \t Pred: {pred}")
        loss = torch.sqrt(loss_fn(pred,true_val ))
#         print(f"loss: {loss.item()}")
        train_loss+=loss.item()
        # Backpropagation
        loss.backward()
        optimizer.step()

        if(i>0 and i % 20 == 0):
            print(f"loss: {train_loss/i:>4f}  [{i:>5d}/{size:>5d}]")
            print(f"true Val: {true_val} \t Pred: {pred}")
    return train_loss

# def test_loop(dataloader, model, loss_fn):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     test_loss, correct = 0, 0

#     with torch.no_grad():
#         for batch,sample in enumerate(dataloader):
#             pred = model(sample['image'].float().cuda())
#             true_val = sample['label'].long()
#             true_val = np.argmax(true_val,axis=1).cuda()
#             test_loss += loss_fn(pred, true_val).item()
#             correct += (1 if torch.equal(np.argmax(pred, axis=1), np.argmax(sample['label'], axis=1)) else 0)
# # (pred == sample['label'])
#     test_loss /= num_batches
#     correct /= size
#     print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

In [ ]:
def train_loop_without_batch(train_data, model, loss_fn, optimizer):
    invMean = [-m/s for (m, s) in zip(MEAN, STD)]
    invStd = [1/s for s in STD]
    # define our de-normalization transform
    deNormalize = transforms.Normalize(mean=invMean, std=invStd)
    train_loss = 0
    for i,sample in enumerate(train_data):
#         print(sample['image'])
#         image = deNormalize(sample['image']).cpu().numpy()
#         image = (image * 255).astype("uint8")
#         image = image.transpose((1, 2, 0))
#         plt.imshow(image)
        pred = model(sample['image'].float().cuda().unsqueeze(0))
#         pred = model(image)
#         print(sample['image'].float().cuda().unsqueeze(0))
        true_val = sample['pwp']
        true_torch = torch.from_numpy(np.asarray(true_val))

#         print(f"true Val: {true_val} \t Pred: {pred}")
#         print(f"Pred: {pred.size()}")

#         print(f"true Val: {true_torch.size()}") 

        loss = torch.sqrt(loss_fn(pred.to(device).float(),true_torch.to(device).float() ))
#         print(f"loss: {loss}")
        train_loss+=loss.item()
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i>0 and i % 200 == 0):
            print(f"\n{i} / {len(train_data)} \n ----------------")
            print(f"Avg Loss :{train_loss/200}")
            print(f"true Val: {true_val} \t Pred: {pred}")
            train_loss=0


In [ ]:
epochs = 70
# train_loss = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    loss = train_loop(train_dataloader, model, loss_fn, optimizer)
#     train_loss.append(loss)
#     test_loop(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
# torch.save(model,"../input")

In [ ]:
class Test_Pet_Data(Dataset):

    def __init__(self, path=None, transform=None):
        self.transform = transform
        self.path = path
        self.img_path = [] 
        for dirname, _, filenames in os.walk(self.path):
            for filename in filenames:
                self.img_path.append(os.path.join(dirname, filename))

        self.transform = transform

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        img_name = os.path.splitext(os.path.relpath(self.img_path[idx],self.path))[0]
#         print(img_name)
#         i = np.where((train['Id'] == img_name))
#         print(f" i:{i[0][0]}")
#         i = i[0][0]
#         pwp = train['Pawpularity'][i] 
#         pwp = pwp/100
        image = io.imread(self.img_path[idx])

        if self.transform:
            trns_image = self.transform(image)
        
        sample = {'image': trns_image, 'name': img_name,}
        return sample

In [ ]:
test_data = Test_Pet_Data(path='/kaggle/input/petfinder-pawpularity-score/test/', transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((512,512)),
    transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(90),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD),
]) )

In [ ]:
for i in range (len(test_data)):
    print(f"val : {test_data[i]['image'].shape}  id: {test_data[i]['name']}")

In [ ]:
df = pd.DataFrame(columns=['Id','Pawpularity' ])

In [ ]:
df

In [ ]:
# test_pred = []
for i,sample in enumerate(test_data):
#         print(sample['image'])
#         image = deNormalize(sample['image']).cpu().numpy()
#         image = (image * 255).astype("uint8")
#         image = image.transpose((1, 2, 0))
#         plt.imshow(image)
        pred = model(sample['image'].float().cuda().unsqueeze(0))
#         print(f"{sample['name']} , {pred.squeeze(0).squeeze(0)}")
#         pred = model(image)
        pred_val = pred.squeeze(0).squeeze(0)
        pred_val = float(pred_val)
        df=df.append({'Id':sample['name'], 'Pawpularity':pred_val},ignore_index=True)

In [ ]:
df

In [ ]:
df.to_csv('submission.csv',index=False)